# Fuzzy Match two dataframes by company name

## Preprocessing

In [24]:
#pip install fuzzywuzzy

In [1]:
import pandas as pd
import openpyxl

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

from tqdm import tqdm

/home/maria/miniconda3/envs/llmSeminar/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
dbpedia = pd.read_csv("../Data/DBPedia_Companies_Preprocessed.csv")
sbti = pd.read_excel("../Data/companies-taking-action.xlsx")
forbes=pd.read_csv("../Data/Forbes_2000_top_company_CLNQ11.csv")

In [3]:
#Create Unique IDs for forbes and sbti dataset
sbti.index = ["SBTI_" + str(i) for i in range(1, len(sbti)+1)] # start at 1 like in the xml file
sbti.index.name = "UID"
sbti = sbti.reset_index()

forbes.index = ["Forbes_" + str(i) for i in range(1, len(forbes)+1)] # start at 1 like in the xml file
forbes.index.name = "UID"
forbes = forbes.reset_index()

In [4]:
# use same column name for company name across datasets

dbpedia["name"] = dbpedia["label"]
sbti["name"] = sbti["Company Name"]
forbes["name"] = forbes["Organization Name"]


## Fuzzy matching

In [113]:
def fuzzymatch_dfs_on_name(df1, df2, min_similarity_score):

    def get_df_name(df):
        """get df name as string"""
        name =[x for x in globals() if globals()[x] is df][0]
        return name

    df1_name = get_df_name(df1) # extract df name to use as column name in resultin df
    df2_name = get_df_name(df2)
    match_name = df1_name + "_" + df2_name

    fuzzy_match_results = []
    match_id = 1

    df1_names = df1["name"].tolist() # list to loop over
    df2_names = df2["name"].tolist()

    for name1 in tqdm(df1_names):
        
        match, score = process.extractOne(name1, df2_names, scorer=fuzz.token_sort_ratio)
        
        if score >= min_similarity_score:
            match_info = {
                "Match ID": match_name + "_" + str(match_id),
                df1_name + "_uid": df1[df1['name'] == name1]['UID'].values[0], # include uids of both companies
                df2_name + "_uid": df2[df2['name'] == match]['UID'].values[0],
                df1_name + "_company": name1, # include names of both companies
                df2_name + "_company": match,
                "Similarity Score": score
            }

            fuzzy_match_results.append(match_info)
            match_id += 1

    fuzzy_match_df = pd.DataFrame(fuzzy_match_results)

    return fuzzy_match_df

run fuzzy matching with each dataset combination:

In [119]:
# dbpedia & forbes

fuzzy_match_dbpedia_forbes = fuzzymatch_dfs_on_name(dbpedia, forbes, 10)
#print(fuzzy_match_dbpedia_forbes.head())

#save
fuzzy_match_dbpedia_forbes.to_excel("../Python/Identiy Resolution/dbp_forbes_fuzzy_match_results.xlsx", index=False)


  0%|          | 0/10720 [00:00<?, ?it/s]

100%|██████████| 10720/10720 [12:16<00:00, 14.56it/s]


In [118]:
# dbpedia & sbti
fuzzy_match_dbpedia_sbti = fuzzymatch_dfs_on_name(dbpedia, sbti, 10)

#print(fuzzy_match_dbpedia_sbti.head())

#save
fuzzy_match_dbpedia_sbti.to_excel("../Python/Identiy Resolution/dbp_sbti_fuzzy_match_min10_results.xlsx", index=False)

  0%|          | 0/10720 [00:00<?, ?it/s]

100%|██████████| 10720/10720 [41:49<00:00,  4.27it/s]


In [115]:
# forbes & sbti
fuzzy_match_forbes_sbti = fuzzymatch_dfs_on_name(sbti, forbes, 10)

#print(fuzzy_match_forbes_sbti.head())

#save
fuzzy_match_forbes_sbti.to_excel("../Python/Identiy Resolution/forbes_sbti_fuzzy_match_min10_results.xlsx", index=False)

100%|██████████| 6157/6157 [08:26<00:00, 12.15it/s]
